# Theme detection probe
Quick check for the viz widget theme detector. Run the two cells below, then toggle your IDE/notebook theme; the live readouts should update without re-running.
- Cell 2 renders the pipeline widget with the inline theme debug badge.
- Cell 3 is a lightweight JS probe that reports the detected background and light/dark decision.


In [ ]:
from hypernodes import Pipeline, node
from hypernodes.viz.visualization_widget import PipelineWidget

@node(output_name="greeting")
def hello(name: str = "theme"):
    return f"hello, {name}"

# Bind the default input so the widget renders without extra args
pipeline = Pipeline(nodes=[hello]).bind(name="theme-check")
PipelineWidget(pipeline, theme="auto", depth=1)


In [ ]:
from IPython.display import HTML

HTML(r"""
<div id="theme-probe" style="font-family: monospace; padding: 10px; border: 1px solid #cbd5e1; border-radius: 8px; margin-top: 8px;">
  Waiting for theme detection...
</div>
<script>
(() => {
  const target = document.getElementById('theme-probe');
  const parseColor = (value) => {
    if (!value) return null;
    const el = document.createElement('div');
    el.style.color = value;
    el.style.display = 'none';
    document.body.appendChild(el);
    const resolved = getComputedStyle(el).color || '';
    el.remove();
    const nums = resolved.match(/[\d\.]+/g);
    if (!nums || nums.length < 3) return null;
    const [r, g, b] = nums.slice(0, 3).map(Number);
    return { luminance: 0.299 * r + 0.587 * g + 0.114 * b, resolved, raw: value };
  };

  const detect = () => {
    const candidates = [];
    const addCandidate = (val, source) => { if (val) candidates.push({ val: val.trim(), source }); };

    try {
      const doc = window.parent?.document;
      if (doc) {
        const root = getComputedStyle(doc.documentElement);
        const body = getComputedStyle(doc.body);
        addCandidate(root.getPropertyValue('--vscode-editor-background'), '--vscode-editor-background');
        addCandidate(body.backgroundColor, 'parent body background');
        addCandidate(root.backgroundColor, 'parent root background');
      }
    } catch (e) {}

    addCandidate(getComputedStyle(document.body).backgroundColor, 'iframe body');

    let chosen = candidates.find(c => parseColor(c.val));
    if (!chosen) chosen = { val: 'transparent', source: 'default' };
    const parsed = parseColor(chosen.val);

    let theme = parsed ? (parsed.luminance > 150 ? 'light' : 'dark') : 'dark';
    let source = parsed ? `${chosen.source} luminance` : chosen.source;

    try {
      const doc = window.parent?.document;
      if (doc) {
        const kind = doc.body.getAttribute('data-vscode-theme-kind');
        if (kind) {
          theme = kind.includes('light') ? 'light' : 'dark';
          source = 'vscode-theme-kind';
        } else if (doc.body.className.includes('vscode-light')) {
          theme = 'light';
          source = 'vscode body class';
        }
      }
    } catch (e) {}

    if (source === 'default' && window.matchMedia) {
      if (window.matchMedia('(prefers-color-scheme: light)').matches) {
        theme = 'light';
        source = 'prefers-color-scheme';
      } else if (window.matchMedia('(prefers-color-scheme: dark)').matches) {
        theme = 'dark';
        source = 'prefers-color-scheme';
      }
    }

    return { theme, background: parsed ? (parsed.resolved || parsed.raw || chosen.val) : chosen.val, luminance: parsed?.luminance ?? null, source };
  };

  const render = () => {
    const state = detect();
    target.innerHTML = `
      <div><strong>auto theme:</strong> ${state.theme}</div>
      <div><strong>background:</strong> ${state.background}</div>
      <div><strong>luminance:</strong> ${state.luminance !== null ? Math.round(state.luminance) : 'n/a'}</div>
      <div><strong>source:</strong> ${state.source}</div>
    `;
  };

  render();
  const observer = new MutationObserver(render);
  try {
    const doc = window.parent.document;
    observer.observe(doc.documentElement, { attributes: true, attributeFilter: ['class', 'data-vscode-theme-kind', 'style']});
    observer.observe(doc.body, { attributes: true, attributeFilter: ['class', 'data-vscode-theme-kind', 'style']});
  } catch (e) {}

  const mq = window.matchMedia ? window.matchMedia('(prefers-color-scheme: dark)') : null;
  const mqHandler = () => render();
  if (mq && mq.addEventListener) mq.addEventListener('change', mqHandler);
  else if (mq && mq.addListener) mq.addListener(mqHandler);
})();
</script>
""")
